<a href="https://colab.research.google.com/github/Flip-data96/Assistente_de_Voz_Multi_Idiomas_Com_Whisper_e_ChatGPT/blob/main/Assistente_de_Voz_Multi_Idiomas_Com_Whisper%2C_ChatGPT_e_Fallback_para_Google_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ==========================================================
# INSTALAÇÃO DE DEPENDÊNCIAS
# ==========================================================

!pip install -q git+https://github.com/openai/whisper.git
!pip install -q openai
!pip install -q google-generativeai
!pip install -q gTTS

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.3/188.3 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 4.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
typer 0.24.0 requires click>=8.2.1, but you have click 8.1.8 which is incompatible.


In [3]:
# ==========================================================
# 🎙️ VOICE AI ASSISTANT
# ==========================================================

import os
import whisper
import openai
import google.generativeai as genai
from gtts import gTTS
from IPython.display import Audio, display, Javascript, HTML
from google.colab import output, userdata
from base64 import b64decode

language = "pt"

# ==========================================================
# 🎤 GRAVAÇÃO DE ÁUDIO
# ==========================================================

RECORD_JS = """
async function recordAudio(seconds) {
    const sleep = time => new Promise(resolve => setTimeout(resolve, time));
    const stream = await navigator.mediaDevices.getUserMedia({ audio: true });
    const recorder = new MediaRecorder(stream);
    let chunks = [];

    recorder.ondataavailable = e => chunks.push(e.data);
    recorder.start();
    await sleep(seconds * 1000);

    return new Promise(resolve => {
        recorder.onstop = async () => {
            const blob = new Blob(chunks);
            const reader = new FileReader();
            reader.readAsDataURL(blob);
            reader.onloadend = () => resolve(reader.result);
        };
        recorder.stop();
    });
}
"""

def record_audio(seconds=5, filename="request_audio.wav"):
    display(Javascript(RECORD_JS))
    print("🎙️ Gravando... Fale agora!")

    js_result = output.eval_js(f"recordAudio({seconds})")
    audio_data = b64decode(js_result.split(",")[1])

    with open(filename, "wb") as f:
        f.write(audio_data)

    print("✅ Gravação finalizada.")
    return filename

# ==========================================================
# 🧠 TRANSCRIÇÃO
# ==========================================================

def transcribe(audio_path):
    print("🧠 Carregando modelo Whisper...")
    model = whisper.load_model("small")

    print("✍️ Transcrevendo...")
    result = model.transcribe(audio_path, fp16=False, language=language)

    return result["text"]

# ==========================================================
# 🤖 LLM (OpenAI → Gemini fallback)
# ==========================================================

def generate_response(prompt):

    try:
        openai_key = os.getenv("OPENAI_API_KEY")
        if openai_key:
            client = openai.OpenAI(api_key=openai_key)
            response = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[{"role": "user", "content": prompt}]
            )
            return response.choices[0].message.content
    except:
        pass

    try:
        google_key = userdata.get("GOOGLE_API_KEY")
        if google_key:
            genai.configure(api_key=google_key)
            model = genai.GenerativeModel("gemini-1.5-flash")
            response = model.generate_content(prompt)
            return response.text
    except:
        pass

    return "Nenhuma A.P.I configurada."

# ==========================================================
# 🔊 SÍNTESE DE VOZ
# ==========================================================

def speak(text, filename="response.mp3"):
    tts = gTTS(text=text, lang=language)
    tts.save(filename)
    return filename

# ==========================================================
# 🚀 PIPELINE COMPLETO
# ==========================================================

def run_assistant():
    print("\n=== INICIANDO ASSISTENTE ===\n")

    audio_file = record_audio(5)
    transcription = transcribe(audio_file)

    print("\n📝 Você disse:")
    print(transcription)

    response = generate_response(transcription)

    print("\n🤖 Resposta:")
    print(response)

    response_audio = speak(response)

    display(Audio(response_audio, autoplay=True))

# ==========================================================
# 🎛️ BOTÃO HTML
# ==========================================================

display(HTML("""
<div style="text-align:center; margin-top:20px;">
    <button onclick="google.colab.kernel.invokeFunction('runAssistant', [], {})"
            style="background-color:#28a745;
                   color:white;
                   font-size:20px;
                   padding:15px 30px;
                   border:none;
                   border-radius:10px;
                   cursor:pointer;">
        🎙️ Gravar
    </button>
</div>
"""))

output.register_callback("runAssistant", lambda: run_assistant())


=== INICIANDO ASSISTENTE ===



<IPython.core.display.Javascript object>

🎙️ Gravando... Fale agora!
✅ Gravação finalizada.
🧠 Carregando modelo Whisper...


100%|████████████████████████████████████████| 461M/461M [00:02<00:00, 206MiB/s]


✍️ Transcrevendo...

📝 Você disse:
 como fazer um pastel



🤖 Resposta:
Nenhuma A.P.I configurada.
